# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [6]:
with open("10-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'12109832101432101234107652158943210178765892\n03078456210145696701218943067654396549456701\n54562364345436789874327832107810387630345210\n65401875696925210765498017656901234521254321\n78078956787814321544567328943217890012189450\n69101045698701055432123410812206921983098765\n43232132509652566785010569701105435674549056\n58943001419143478994321678983210304105678143\n67653214328012988765690787894321213289437212\n45654301037001089650787096765010034576524301\n56789890156121072341256101896654123678915498\n43276765243234561212345234987783210569206789\n54109854312789870109012345676898765454106543\n45610123203650105438721056765609674323287012\n54781010154543216521635489832014589210398013\n67898543269854107610544376541023008101296323\n54987656576765678923455210458782112010387456\n23122189983454989012966904349698103465456567\n12033078012763210101877813234521098578956798\n03944565430887654012109320121034787632347897\n87856556021991047121238458945695698961036016\n965874321108723389303475672327800

In [20]:
test_data_raw = r"""89010123
78121874
87430965
96549874
45678903
32019012
01329801
10456732"""

test_data_raw2 = r"""0123
1234
8765
9876"""

def preprocess_data (data):
    # dtype='U10'
    
    # dtype='U10'
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype=int)



test_data = preprocess_data(test_data_raw)
test_data2 = preprocess_data(test_data_raw2)
display(test_data)

array([[8, 9, 0, 1, 0, 1, 2, 3],
       [7, 8, 1, 2, 1, 8, 7, 4],
       [8, 7, 4, 3, 0, 9, 6, 5],
       [9, 6, 5, 4, 9, 8, 7, 4],
       [4, 5, 6, 7, 8, 9, 0, 3],
       [3, 2, 0, 1, 9, 0, 1, 2],
       [0, 1, 3, 2, 9, 8, 0, 1],
       [1, 0, 4, 5, 6, 7, 3, 2]])

In [7]:
full_data = preprocess_data(data_raw)
full_data

array([[1, 2, 1, ..., 8, 9, 2],
       [0, 3, 0, ..., 7, 0, 1],
       [5, 4, 5, ..., 2, 1, 0],
       ...,
       [4, 0, 1, ..., 3, 4, 3],
       [3, 0, 0, ..., 5, 0, 1],
       [2, 1, 0, ..., 4, 3, 2]])

In [40]:
def solution (data, verbose=False):
    zeros = np.argwhere(data == 0)
    nines = np.argwhere(data == 9)

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])
    
    def isNext(array: np.ndarray, source: tuple, target: tuple):
        if array[source] == array[target] - 1:
            return True
        else:
            return False
    
    
    def build_dir_graph(array):
        graph = dict()
        for row in range(array.shape[0]):
            for col in range(array.shape[1]):
                directions = [(1,0), (-1,0), (0,1), (0,-1)]
                next_nodes = [(row+direction[0], col+direction[1])  for direction in directions 
                                                                        if isValid(array.shape, (row+direction[0], col+direction[1])) 
                                                                            and isNext(array, (row, col), (row+direction[0], col+direction[1])) ]
                if next_nodes:
                    graph[(row, col)] = next_nodes
        return graph
                    
    def DFS (graph, array, source):
        leafs = []
        queue = [source]
        
        while queue:
            next = queue.pop(0)

            if next in graph:
                queue = graph[next] + queue 
            elif array[next] == 9:
                leafs.append(next)
        return leafs
                
    
    
    graph = build_dir_graph(data)
    total = 0
    for start in [tuple(e) for e in zeros.tolist()]:
        leafs = DFS (graph, data, start)
        total += len(set(leafs))
        

    return total



sol = solution(test_data, verbose=True)

print(sol)


36


In [39]:
sol = solution(full_data)
print(sol)

582


# Part 2

In [43]:
def solution2 (data, verbose=False):

    zeros = np.argwhere(data == 0)
    nines = np.argwhere(data == 9)

    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])
    
    def isNext(array: np.ndarray, source: tuple, target: tuple):
        if array[source] == array[target] - 1:
            return True
        else:
            return False
    
    
    def build_dir_graph(array):
        graph = dict()
        for row in range(array.shape[0]):
            for col in range(array.shape[1]):
                directions = [(1,0), (-1,0), (0,1), (0,-1)]
                next_nodes = [(row+direction[0], col+direction[1])  for direction in directions 
                                                                        if isValid(array.shape, (row+direction[0], col+direction[1])) 
                                                                            and isNext(array, (row, col), (row+direction[0], col+direction[1])) ]
                if next_nodes:
                    graph[(row, col)] = next_nodes
        return graph
                    
    def DFS (graph, array, source):
        leafs = []
        queue = [source]
        
        while queue:
            next = queue.pop(0)

            if next in graph:
                queue = graph[next] + queue 
            elif array[next] == 9:
                leafs.append(next)
        return leafs
                
    
    
    graph = build_dir_graph(data)
    total = 0
    for start in [tuple(e) for e in zeros.tolist()]:
        leafs = DFS (graph, data, start)
        total += len(leafs)
        

    return total




sol = solution2(test_data, verbose=True)
print(sol)
    

81


In [44]:
sol = solution2(full_data)
print(sol)

1302
